In [1]:
import sys
sys.path.append('../')

In [2]:
from mypackage import *

In [3]:
import yfinance as yf

In [4]:
#Get the data
ticker = "AAPL"  # Apple Inc. as an example
stock_data = yf.download(ticker, period="2y", interval="1d")

# Extract high, low, close, and volume from the stock data
highs = stock_data['High'].values
lows = stock_data['Low'].values
closes = stock_data['Close'].values
volumes = stock_data['Volume'].values

[*********************100%***********************]  1 of 1 completed


In [9]:
indicators_df = calculate_technical_indicators(highs, lows, closes, volumes) # The minimum price array length to run this is >50
indicators_df = indicators_df.dropna() # Make sure the indicators are well aligned and don't contain Nan's at the end

In [10]:
indicators_df

,SMA,EMA,RSI,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower,ATR,VWAP,CCI,...,TSI,Chaikin_Oscillator,Ichimoku_Conversion_Line,Ichimoku_Base_Line,Keltner_Upper,Keltner_Lower,Parabolic_SAR,OBV,AD_Line,Aroon_Oscillator
49,133.397000,136.745841,93.242029,1.396013,-0.447310,144.909419,121.884581,3.492856,138.639742,148.094723,...,-21.833349,9.571221e+07,139.360001,134.490002,147.104022,126.387659,132.359335,-4.175500e+06,6.330771e+07,-56.0
50,134.391499,137.620522,92.319686,1.918428,0.025838,146.605007,122.177992,3.403570,138.773672,152.795937,...,-19.767616,1.081785e+08,140.570000,135.474995,147.761519,127.479524,133.208562,6.638030e+07,8.965970e+07,-60.0
51,135.060999,138.132853,79.649322,2.072134,0.435097,147.615901,122.506098,3.352854,138.842962,115.456323,...,-18.254797,9.755495e+07,140.570000,135.474995,147.878989,128.386718,134.023819,2.365000e+06,3.275693e+07,-60.0
52,135.778999,138.719247,80.294935,2.271851,0.802448,148.714720,122.843278,3.275711,138.933334,101.445570,...,-16.286249,8.889846e+07,140.599998,135.474995,148.286295,129.152199,134.806466,6.823950e+07,9.543344e+07,-60.0
53,136.796999,139.358366,78.757244,2.493374,1.140633,149.401736,124.192262,3.435711,139.057973,100.565576,...,-13.967771,7.522834e+07,141.899994,135.474995,148.626657,130.090075,135.557807,1.459031e+08,1.384491e+08,-60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,229.996000,227.885986,32.646304,-0.985059,0.466079,238.796271,221.195729,3.941428,180.657379,-124.482465,...,7.898433,-1.507581e+07,227.839996,229.084999,375.614606,80.157366,231.252886,1.550584e+09,1.355521e+09,-40.0
498,229.918000,227.847321,37.070131,-0.870339,0.198795,238.778231,221.057769,4.022143,180.722943,-58.438458,...,8.080489,-2.806724e+06,227.839996,229.084999,384.004651,71.689990,230.098598,1.592721e+09,1.387474e+09,-40.0
499,229.888500,227.762814,33.072544,-0.812021,-0.003368,238.783930,220.993070,4.011428,180.781760,-44.461884,...,8.289305,-1.112541e+06,227.839996,229.084999,390.092643,65.432986,229.059739,1.554392e+09,1.367884e+09,-40.0
500,229.535000,227.426356,30.764121,-0.974856,-0.197665,238.734679,220.335321,4.124286,180.842247,-93.342375,...,8.994720,9.510313e+06,226.055000,229.084999,393.861861,60.990850,228.124766,1.512387e+09,1.380485e+09,40.0


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-14,148.970001,150.279999,147.429993,148.279999,146.736923,73374100
2022-11-15,152.220001,153.589996,148.559998,150.039993,148.478607,89868300
2022-11-16,149.130005,149.869995,147.289993,148.789993,147.241638,64218300
2022-11-17,146.429993,151.479996,146.149994,150.720001,149.151566,80389400
2022-11-18,152.309998,152.699997,149.970001,151.289993,149.715576,74829600
...,...,...,...,...,...,...
2024-11-06,222.610001,226.070007,221.190002,222.720001,222.475235,54561100
2024-11-07,224.630005,227.880005,224.570007,227.479996,227.229996,42137700
2024-11-08,227.169998,228.660004,226.410004,226.960007,226.960007,38328800
